Задание практика №2:  обучить классификатор и классифицировать тексты, используя для представления документов вектора полученные с помощью LSA (латентно-семантический анализ).  В случае ЛСА используется матрица терм-документ, ячейки матрицы содержат вес TF-IDF каждого терма (слова) в соответствующем документе. До построения матрицы тексты нужно предобратотать. Подсказка ->

# LSI model

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.datasets import fetch_20newsgroups
number_of_topic = 3
newsgroups_train = fetch_20newsgroups(subset='train')
indices = np.where((newsgroups_train.target >= 0) & (newsgroups_train.target <= number_of_topic))[0]
texts = [newsgroups_train.data[i] for i in indices]
target = np.array([newsgroups_train.target[i] for i in indices])
print(f"Всего элементов: {target.shape[0]}")
for i in range(number_of_topic):
    count = np.count_nonzero(target == i)
    print(f"ко-во элементов из класса {i}: {count}")

Всего элементов: 2245
ко-во элементов из класса 0: 480
ко-во элементов из класса 1: 584
ко-во элементов из класса 2: 591


In [13]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [14]:
def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_tokens


def preprocess_text(text): 
    # Удаление пунктуации
    text = re.sub(r'[^\w\s]', '', text)
    
    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    
    tokens = [token for token in tokens if token not in stop_words]

    lemmatized_tokens = lemmatize_tokens(tokens)
    
    return " ".join(lemmatized_tokens)

In [15]:
texts[0][:60]

'From: jgreen@amber (Joe Green)\nSubject: Re: Weitek P9000 ?\nO'

In [16]:
corpus = [preprocess_text(i) for i in texts]
corpus[0][:60]

'From jgreenamber Joe Green Subject Re Weitek P9000 Organizat'

In [17]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [41]:
tfidf_matrix.shape

(2245, 50716)

In [47]:
mini_size = None
for i in range(2246, 500, -150):
    svd = TruncatedSVD(n_components=i)
    
    X_approx = svd.fit_transform(tfidf_matrix)
    
    # Вычисление нормы tfidf_matrix
    tfidfm_norm = sp.linalg.norm(tfidf_matrix, ord=np.inf)
    
    X_approx_inverse = svd.inverse_transform(X_approx)
    
    error = np.linalg.norm(tfidf_matrix - X_approx_inverse) / tfidfm_norm
    if error <= 0.25:
        mini_size = i
    else:
        break

In [48]:
mini_size

1646

In [50]:
lsa_model = TruncatedSVD(n_components=mini_size)  # Выбор количества компонентов
lsa_vectors = lsa_model.fit_transform(tfidf_matrix)

## Обучение классификатора

In [57]:
classifier = DecisionTreeClassifier()  

X_train, X_test, y_train, y_test = train_test_split(lsa_vectors, target, test_size=0.2, random_state=42)

param_grid = {'max_depth': range(1, 10, 1)}

grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Лучшая глубина дерева для {grid_search.best_params_['max_depth']}")
print(f"значение F1-score:{f1}")

Лучшая глубина дерева для 6
значение F1-score:0.7916930308972283
